In [1]:
import os
os.environ['JAVA_HOME'] = 'C:/Program Files/Java/jdk1.8.0_291'

In [2]:
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf())

In [3]:
import numpy as np
data = sc.textFile("q2/data/user-shows.txt")
user = data.map(lambda line: np.array([int(value) for value in line.split()]))

In [4]:
R = np.array(user.collect())
P = []
Q = []
alex = []
for i in range(len(R)):
    s = R[i].sum()
    if(s != 0):
        s = s**-0.5
    P.append(s)
for j in range(len(R[0])):
    s = R[:,j].sum()
    if(s != 0):
        s = s**-0.5
    Q.append(s)
P = np.diag(P)
Q = np.diag(Q)

In [5]:
Si = Q.dot(R.T.dot(R.dot(Q)))
Su = P.dot(R.dot(R.T.dot(P)))

In [143]:
fav1 = sc.parallelize(R.dot(Si)[499,:100]).zipWithIndex().map(lambda t:(t[1],t[0]))
fav2 = sc.parallelize(Su.dot(R)[499,:100]).zipWithIndex().map(lambda t:(t[1],t[0]))

In [144]:
shows = sc.textFile("q2/data/shows.txt").zipWithIndex().map(lambda t:(t[1],t[0]))

In [150]:
fav1.join(shows).sortBy(lambda x:(-x[1][0],x[0])).values().take(5)

[(31.364701678342406, '"FOX 28 News at 10pm"'),
 (30.001141798877768, '"Family Guy"'),
 (29.396797773402554, '"NBC 4 at Eleven"'),
 (29.227001561500487, '"2009 NCAA Basketball Tournament"'),
 (28.971277674055564, '"Access Hollywood"')]

In [151]:
fav2.join(shows).sortBy(lambda x:(-x[1][0],x[0])).values().take(5)

[(908.4800534761282, '"FOX 28 News at 10pm"'),
 (861.1759992873298, '"Family Guy"'),
 (827.6012954743584, '"2009 NCAA Basketball Tournament"'),
 (784.7819589039742, '"NBC 4 at Eleven"'),
 (757.6011181024229, '"Two and a Half Men"')]